# OpEn Rust Examples: Obstacle-avoiding Path Planning

In this example, we are going to implement a path planner for a mobile robot. We will consider the robot's next position as its decision variable, rather than using its control input. Note that, in this example, we do not consider the robot's dynamics. 


### Prerequisite

- [Obstacle avoidance constraint](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_obs_avoidance_simplified.ipynb)
- [A generalised gradient function](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_general_diff.ipynb)



## Problem Formulation

Minimise 
$$ f(\mathbf{x}) = \sum_{t=0}^{T_H} (x_t - x_{ref})^{\top}Q_t(x_t - x_{ref}) $$
<div style="text-align: right"> (P1) </div>

where   
- $ \mathbf{x} = \{x_0, x_1, ..., x_{T_H} \} $: the decision variables over the time horizon $T_H$ (i.e. the planned trajectory)
- $x_t \in \mathbb{R}^2$: the 2-d position at time $t$. 
- $x_{ref}$: the goal position. 

subject to 
$$ x_0 = x_{start}$$
<div style="text-align: right"> (C0) </div>

$$ \|x_{t+1} - x_t \| \le u_{max}, \quad\quad k \in \{0,...,T_H-1\}$$
<div style="text-align: right"> (C1) </div>


$$ \psi(\mathbf{x}) =   \sum_{j=1}^{n_{o}}[1 - (x_t - c_j)^{\top}(x_t - c_j)]_{+} = 0, \quad\quad k \in \{0,...,T_H-1\}$$
<div style="text-align: right"> (C2) </div>

$\psi(\mathbf{x})$ becomes $0$ if all $x_t$ is outside every circular shaped obstacle $j$ (nb: $c_j$ is its centre position); and increases in the interior of it as $x_t$ moves into the obstacle from its boundary. 

### Reduced Problem

- In this example, we only consider a single circular shape obstacle (i.e. $n_o = 1$)
- We simply consider the time horizon $T_H = 1$, and neglect $Q_t$. 

Thus, the original problem can be reduced to:

Minimise 
$$ f(x_t) = (x_t - x_{ref})^{\top}(x_t - x_{ref}) $$
<div style="text-align: right"> (P1) </div>


subject to 
$$ x_0 = x_{start}$$
<div style="text-align: right"> (C0) </div>

$$ [\|x_{t} - x_0 \| - u_{max}]_{+} = 0$$
<div style="text-align: right"> (C1) </div>


$$ \psi(\mathbf{x}) =  [1 - (x_t - c)^{\top}(x_t - c)]_{+} = 0$$
<div style="text-align: right"> (C2) </div>



## OpEn Implementation

First, we need to import "optimization_engine". Also, we use "nalgebra" for linear algebra calculation. 
- In your local PC, it should be also declared in "Cargo.toml".
- Instead, in this jupyter notebook, we need to have "extern crate" as follows. 


In [2]:
extern crate optimization_engine;
use optimization_engine::{
    alm::*,
    constraints::*, panoc::*, *
};
extern crate nalgebra;
use nalgebra::base::{*};
// use nalgebra::base::{Matrix4, Matrix4x2, Matrix4x1};
// use std::cmp;

### Problem Master Class

You should note that `AlmFactory` should have `f` and `df` being with `
fn f(u: &[f64], cost: &mut f64) -> Result<(), SolverError>` and `fn df(u: &[f64], grad: &mut [f64]) -> Result<(), SolverError>`, respectively. It means that it could be nicer if we have a master class that can simply turn out `f` or `df` values. Such an architecture is used in [`relaxed_ik` rust version](https://github.com/uwgraphics/relaxed_ik/blob/dev/src/RelaxedIK_Rust/src/bin/lib/groove/objective_master.rs), which is a good example to be worth having a look. In this example as well, we are going to implement a problem master class as follows. 


In [3]:
pub struct ProblemMaster{
    x_now: Matrix2x1<f64>, // Robot Start Position
    x_ref: Matrix2x1<f64>, // Robot Goal Position
    x_obs: Matrix2x1<f64>, // Obstacle Position
    u_max: f64, // Dyanmics Radius
}

impl ProblemMaster{
    pub fn init(_x_start: Matrix2x1<f64>, _x_ref: Matrix2x1<f64>, _x_obs: Matrix2x1<f64>, _u_max: f64) -> Self {
        let x_now = _x_start;
        let x_ref = _x_ref;
        let x_obs = _x_obs;
        let u_max = _u_max;
        Self{x_now, x_ref, x_obs, u_max}            
    }
    
    
    // ========= Cost function (You need to modify this) =========
    pub fn f_call(&self, u: &[f64]) -> f64{        
        let cost = (u[0]-self.x_ref[(0,0)]).powi(2) + (u[1]-self.x_ref[(1,0)]).powi(2);
        cost
    }
    // ===========================================================
    
    pub fn f(&self, u: &[f64], cost: &mut f64){
        *cost = self.f_call(u);        
    }
    
    // Gradient of the cost function
    pub fn df(&self, u: &[f64], grad: &mut [f64]){
        let mut f_0 = self.f_call(u);
        
        for i in 0..u.len() {
            let mut u_h = u.to_vec();
            u_h[i] += 0.000001;
            let f_h = self.f_call(u_h.as_slice());
            grad[i] = (-f_0 + f_h) / 0.000001;
        }
       
    } 
    
    // ========= F1 Constraint (You need to modify this) =========
    pub fn f1_call(&self, u: &[f64])-> Vec<f64> {
        let mut f1u = vec![0.0; u.len()];
        // Obstacle Avoidance Constraint (C2)
        f1u[0] = (1.0 - (u[0]-self.x_obs[(0,0)]).powi(2) - (u[1]-self.x_obs[(1,0)]).powi(2) ).max(0.0);
        
        // Dynamics Constraint (C1)
        let mut u_now = Matrix2x1::new(0.0, 0.0);
        for i in 0..u.len(){
            u_now[(i,0)] = u[i];
        }
        let delta = u_now - self.x_now;
        f1u[1] = (delta.norm() - self.u_max).max(0.0);
        
        return f1u;
    }
    // ===========================================================
    
    pub fn f1(&self, u: &[f64], f1u: &mut [f64]){
        let mut f1u_vec = self.f1_call(u); 
        for i in 0..f1u_vec.len(){
            f1u[i] = f1u_vec[i];
        }
    }    
    
    // Jacobian of F1
    pub fn jf1_call(&self, u: &[f64])-> Matrix2<f64> {
        let mut jf1 = Matrix2::new(0.0, 0.0,
                              0.0, 0.0);
        
        let mut f1_0 = self.f1_call(u);

        for i in 0..f1_0.len(){
            for j in 0..u.len() {
                let mut u_h = u.to_vec();
                u_h[j] += 0.000001;
                let f_h = self.f1_call(u_h.as_slice());
                jf1[(i,j)] = (-f1_0[i] + f_h[i]) / 0.000001;
            }                        
        }

        return jf1;        
    } 
    
    // Jacobian Product (JF_1^{\top}*d)
    pub fn f1_jacobian_product(&self, u: &[f64], d: &[f64], res: &mut [f64]){
        let test = self.f1_call(u);
        
        let mut jf1_matrix = self.jf1_call(u);
        if test[0] < 0.0{ // Outside the obstacle
            jf1_matrix[(0,0)] = 0.0;
            jf1_matrix[(0,1)] = 0.0;  
        }          
        
        let mut d_matrix = Matrix2x1::new(0.0, 0.0);
        for i in 0..d.len(){
            d_matrix[(i,0)] = d[i];
        }
        
        let mut res_matrix =  jf1_matrix.transpose()*d_matrix;
        
        res[0] = res_matrix[(0,0)];
        res[1] = res_matrix[(1,0)];  
    }
    
}

### Optimisation function for each time step


In [4]:
fn main_loop(_x_now: &[f64], _x_ref: &[f64], _x_obs: &[f64], _u_max: f64) -> Vec<f64> {
 
    /// ===========================================
    let mut x_now = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_now.len(){
        x_now[(i,0)] = _x_now[i];
    }
    
    let mut x_ref = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_ref.len(){
        x_ref[(i,0)] = _x_ref[i];
    }
    
    let mut x_obs = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_obs.len(){
        x_obs[(i,0)] = _x_obs[i];
    }
    
    let mut u_max = _u_max;        
    
    let mut pm = ProblemMaster::init(x_now, x_ref, x_obs, u_max);
    
    /// ===========================================
    
    let tolerance = 1e-5;
    let nx = 2; // problem_size: dimension of the decision variables
    let n1 = 2; // range dimensions of mappings F1
    let n2 = 0; // range dimensions of mappings F2
    let lbfgs_mem = 5; // memory of the LBFGS buffer
    
    // PANOCCache: All the information needed at every step of the algorithm
    let panoc_cache = PANOCCache::new(nx, tolerance, lbfgs_mem);
    
    // AlmCache: A cache structure that contains all the data 
    // that make up the state of the ALM/PM algorithm
    // (i.e., all those data that the algorithm updates)
    let mut alm_cache = AlmCache::new(panoc_cache, n1, n2);

    let set_c = Zero::new(); // Set C
    let bounds = Ball2::new(None, 1e12); // Set U
    let set_y = Ball2::new(None, 1e12);  // Set Y

    // ============= 
    // Re-define the functions linked to user parameters
    let f = |u: &[f64], cost: &mut f64| -> Result<(), SolverError> {
        pm.f(u, cost);
        Ok(())
    };
    
    let df = |u: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
        pm.df(u, grad);
        Ok(())
    };
    
    let f1 = |u: &[f64], f1u: &mut [f64]| -> Result<(), SolverError> {
        pm.f1(u, f1u);
        Ok(())
    };    
    
    let f1_jacobian_product = |u: &[f64], d: &[f64], res: &mut [f64]| -> Result<(), SolverError> {
        pm.f1_jacobian_product(u,d,res);
        Ok(())
    };      
    // ==============
    
    // AlmFactory: Prepare function psi and its gradient 
    // given the problem data such as f, del_f and 
    // optionally F_1, JF_1, C, F_2
    let factory = AlmFactory::new(
        f, // Cost function
        df, // Cost Gradient
        Some(f1), // MappingF1
        Some(f1_jacobian_product), // Jacobian Mapping F1 Trans
        NO_MAPPING, // MappingF2
        NO_JACOBIAN_MAPPING, // Jacobian Mapping F2 Trans
        Some(set_c), // Constraint set
        n2,
    );

    // Define an optimisation problem 
    // to be solved with AlmOptimizer
    let alm_problem = AlmProblem::new(
        bounds,
        Some(set_c),
        Some(set_y),
        |u: &[f64], xi: &[f64], cost: &mut f64| -> Result<(), SolverError> {
            factory.psi(u, xi, cost)
        },
        |u: &[f64], xi: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
            factory.d_psi(u, xi, grad)
        },
        Some(f1),
        NO_MAPPING,
        n1,
        n2,
    );

    let mut alm_optimizer = AlmOptimizer::new(&mut alm_cache, alm_problem)
        .with_delta_tolerance(1e-5)
        .with_max_outer_iterations(200)
        .with_epsilon_tolerance(1e-6)
        .with_initial_inner_tolerance(1e-2)
        .with_inner_tolerance_update_factor(0.5)
        .with_initial_penalty(100.0)
        .with_penalty_update_factor(1.05)
        .with_sufficient_decrease_coefficient(0.2)
        .with_initial_lagrange_multipliers(&vec![5.0; n1]);

    let mut u = _x_now.to_vec(); // vec![0.0; nx]; // Initial guess
    let solver_result = alm_optimizer.solve(&mut u);
    let r = solver_result.unwrap();
//     println!("\n\nSolver result : {:#.7?}\n", r);
//     println!("Solution u = {:#.6?}", u);
    
    return u;
    
}

### Main Function Generating a trajectory 



In [6]:
fn main(_x_now: &[f64], _x_ref: &[f64], _x_obs: &[f64], _u_max: f64) {
    
//     println!("Solution u = {:#.6?}", _x_now);
    let mut done = false;
    let mut x_now = _x_now.to_vec();
    while !done{
//     for i in 0..60{
        println!("Solution x_now = {:#.6?}   {:#.6?}", x_now[0], x_now[1]);
//         println!("Solution _x_ref = {:#.6?}", _x_ref);
//         println!("Solution _x_obs = {:#.6?}", _x_obs);
        let mut x_next = main_loop(x_now.as_slice(), _x_ref, _x_obs, _u_max);
//         println!("Solution x_next = {:#.6?}", x_next);
        x_now = x_next;
        let delta = ((x_now[0]-_x_ref[0]).powi(2)+(x_now[1]-_x_ref[1]).powi(2)).sqrt();
        if delta < 0.00001{
            done = true;
        }
       

    }
    println!("Solution x_now = {:#.6?}   {:#.6?}", x_now[0], x_now[1]);
       
//     }
        
    
}


### Result

#### Case (1) : $x_{start} = [0.0, 1.0]$;  $x_{ref} = [10.0, 10.0]$; $c = [5.0, 5.0]$; and $u_{max} = 0.1$. 

In [7]:
main(&[0.0, 1.0], &[10.0, 10.0],&[5.0, 5.0], 0.1_f64);

Solution x_now = 0.000000   1.000000
Solution x_now = 0.074334   1.066903
Solution x_now = 0.148670   1.133806
Solution x_now = 0.223004   1.200710
Solution x_now = 0.297335   1.267613
Solution x_now = 0.371669   1.334518
Solution x_now = 0.446002   1.401423
Solution x_now = 0.520337   1.468328
Solution x_now = 0.594670   1.535234
Solution x_now = 0.669004   1.602140
Solution x_now = 0.743337   1.669043
Solution x_now = 0.817669   1.735945
Solution x_now = 0.892002   1.802848
Solution x_now = 0.966335   1.869750
Solution x_now = 1.040669   1.936653
Solution x_now = 1.115004   2.003555
Solution x_now = 1.189338   2.070457
Solution x_now = 1.263670   2.137360
Solution x_now = 1.338002   2.204262
Solution x_now = 1.412335   2.271166
Solution x_now = 1.486667   2.338068
Solution x_now = 1.560998   2.404970
Solution x_now = 1.635330   2.471872
Solution x_now = 1.709664   2.538779
Solution x_now = 1.783994   2.605683
Solution x_now = 1.858325   2.672586
Solution x_now = 1.932657   2.739489
S

The result can be visualised as follows. 

![](https://raw.githubusercontent.com/inmo-jang/optimisation_tutorial/master/tools_examples/OpEn/examples_rust/example_08_pathplanning/path_result.svg)

#### Case (2) : $x_{start} = [1.0, 0.0]$;  $x_{ref} = [10.0, 10.0]$; $c = [5.0, 5.0]$; and $u_{max} = 0.1$. 

In [8]:
main(&[1.0, 0.0], &[10.0, 10.0],&[5.0, 5.0], 0.1_f64);

Solution x_now = 1.000000   0.000000
Solution x_now = 1.066903   0.074334
Solution x_now = 1.133806   0.148670
Solution x_now = 1.200710   0.223004
Solution x_now = 1.267613   0.297335
Solution x_now = 1.334518   0.371669
Solution x_now = 1.401423   0.446002
Solution x_now = 1.468328   0.520337
Solution x_now = 1.535234   0.594670
Solution x_now = 1.602140   0.669004
Solution x_now = 1.669043   0.743337
Solution x_now = 1.735945   0.817669
Solution x_now = 1.802848   0.892002
Solution x_now = 1.869750   0.966335
Solution x_now = 1.936653   1.040669
Solution x_now = 2.003555   1.115004
Solution x_now = 2.070457   1.189338
Solution x_now = 2.137360   1.263670
Solution x_now = 2.204262   1.338002
Solution x_now = 2.271166   1.412335
Solution x_now = 2.338068   1.486667
Solution x_now = 2.404970   1.560998
Solution x_now = 2.471872   1.635330
Solution x_now = 2.538779   1.709664
Solution x_now = 2.605683   1.783994
Solution x_now = 2.672586   1.858325
Solution x_now = 2.739489   1.932657
S

![](https://raw.githubusercontent.com/inmo-jang/optimisation_tutorial/master/tools_examples/OpEn/examples_rust/example_08_pathplanning/path_result2.svg)

## Discussion

- The current version planner may not be able to find a collision-avoided path, depending on the given situation. For example, if we start from (0.0, 0.0), the robot would be stuck at around (4.xx, 4.xx), which is the closest point of the obstacle boundary to the start point, without being able to look for any detouring path. That may be because of the current objective function (i.e. norm-2) or the current time horizon being just one time step. 

- **NOTE**: [The source file of this example](https://github.com/inmo-jang/optimisation_tutorial/tree/master/tools_examples/OpEn/examples_rust/example_08_pathplanning) is slightly different than here. Using the source file, you can plot the graphs above. 





## Future work

- Let's make the `ProblemMaster` have resizable parameters (e.g. not binding to `Matrix2x1`). 

- Let's consider multiple obstacles. 

- Let's consider a longer time horizon. 

- Let's consider a system dynamics. 


